# NOAA 1158: Cooling Model
Simulation of loops in AR NOAA 1158 in which the loops are essentially only allowed to cool, i.e. they are heated by a single pulse at $t=0$ s and then the simulation is run for a sufficiently long enough time so that even the longest loops can cool and drain back to their equilibrium state.

We will use the base field model that we've already built for NOAA 1158, AR #2 in Warren et al. (2012)

In [ ]:
import os
import glob
import warnings
import multiprocessing
import subprocess
warnings.filterwarnings('ignore')

import numpy as np
from sunpy.map import Map
import yt
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn.apionly as sns
import dask.distributed

import synthesizAR
from synthesizAR.model_ext import EbtelInterface,UniformHeating,calculate_free_energy
from synthesizAR.instruments import InstrumentSDOAIA

%matplotlib inline

First, start up a dask scheduler and worker on the localhost.

In [ ]:
client = dask.distributed.Client()
client

## Build Field

In [ ]:
field = synthesizAR.Skeleton.restore('/data/datadrive2/systematic_ar_study/base_noaa1158')

## Heating Model and Loop Configuration

For the loop simulations, we'll use a single pulse of duration 200 s at $t=0$ s with total energy of 

$$
E_s = \frac{(\epsilon B_s)^2}{8\pi}
$$

where $B_s$ is the average field strength along a strand $s$.

Setup all of our loop models using this single-event/cooling model.

In [ ]:
heating_options = {
    'duration':200,
    'average_waiting_time':8800,
    'duration_rise':100,
    'duration_decay':100,
    'stress_level':0.1
}
heating_model = UniformHeating(heating_options)

In [ ]:
ih = synthesizAR.util.InputHandler('/home/wtb2/Documents/codes/ebtelPlusPlus/config/ebtel.example.cfg.xml')
base_config = ih.lookup_vars()
base_config['c1_cond0'] = 6.0
base_config['total_time'] = 1e4
base_config['use_adaptive_solver'] = True
base_config['use_flux_limiting'] = True
base_config['calculate_dem'] = False
base_config['heating']['partition'] = 1.0
base_config['heating']['background'] = 1e-6
base_config['force_single_fluid'] = False
base_config['tau_max'] = 200.0

In [ ]:
ebtel_interface = EbtelInterface(base_config,heating_model,
                                 '/data/datadrive2/systematic_ar_study/noaa1158_cooling/hydro_config/',
                                 '/data/datadrive2/systematic_ar_study/noaa1158_cooling/hydro_results/')

In [ ]:
field.configure_loop_simulations(ebtel_interface)

Now run all of our models and load them back into the field.

In [ ]:
def ebtel_runner(loop):
    subprocess.call([os.path.join('/home/wtb2/Documents/codes/','ebtelPlusPlus/bin/ebtel++.run'),
                     '-c',loop.hydro_configuration['config_filename']])

In [ ]:
pool = multiprocessing.Pool()

In [ ]:
runs = pool.map_async(ebtel_runner,field.loops)
runs.wait()

In [ ]:
field.load_loop_simulations(ebtel_interface,
                            savefile='/data/datadrive2/systematic_ar_study/noaa1158_cooling/loop_parameters.h5')

And save the field.

In [ ]:
field.save('/data/datadrive2/systematic_ar_study/noaa1158_cooling/field_checkpoint')

Peek at the hydrodynamic simulation results.

In [ ]:
fig,axes = plt.subplots(2,1,figsize=(20,10),sharex=True)
plt.subplots_adjust(hspace=0.)
for loop in field.loops[::10]:
    axes[0].plot(loop.time,loop.electron_temperature[:,0].to(u.MK),color=sns.color_palette('deep')[0],alpha=0.01)
    axes[0].plot(loop.time,loop.ion_temperature[:,0].to(u.MK),color=sns.color_palette('deep')[2],alpha=0.01)
    axes[1].plot(loop.time,loop.density[:,0]/1e9,color=sns.color_palette('deep')[0],alpha=0.05)
axes[0].set_xlim([0,base_config['total_time']])
axes[0].set_ylabel(r'$T$ [MK]')
axes[1].set_ylabel(r'$n$ [10$^9$ cm$^{-3}$]')
axes[1].set_xlabel(r'$t$ [s]')

Restore the field here if necessary.

In [ ]:
field = synthesizAR.Skeleton.restore('/data/datadrive2/systematic_ar_study/noaa1158_cooling/field_checkpoint/')

## Instrument Intensity Synthesis

In [ ]:
aia = InstrumentSDOAIA([0,5000]*u.s)

In [ ]:
observer = synthesizAR.Observer(field,[aia])

In [ ]:
observer.build_detector_files('/data/datadrive2/systematic_ar_study/noaa1158_cooling/',
                              ds=field._convert_angle_to_length(1.2*u.arcsec))

In [ ]:
flatten_dag = observer.flatten_detector_counts()

In [ ]:
flatten_dag['SDO_AIA'].compute()

In [ ]:
status = client.compute(flatten_dag['SDO_AIA'])

In [ ]:
status.status

## Property Evaluation Test

In [ ]:
test = [('/data/datadrive2/systematic_ar_study/noaa1158_cooling/tmp_parallel_files/SDO_AIA/94/foofoo.npy','bar') for _ in range(100000)]

In [ ]:
@dask.delayed
def mean_loop_etemperature(electron_temperature):
    return electron_temperature.value.mean()
@dask.delayed
def mean_of_means(means):
    return np.array(means).mean()

In [ ]:
mean_list = [mean_loop_etemperature(make_property_dask_lazy(loop,'electron_temperature')) for loop in field.loops]
all_loop_mean = mean_of_means(mean_list)

In [ ]:
all_loop_mean.compute()

In [ ]:
class Foo(object):
    def __init__(self,n):
        self.n = n
    @property
    def rand_property(self):
        print('Evaluating property')
        return np.random.rand(self.n)

    def rand_method(self):
        print('Evaluating Method')
        return np.random.rand(self.n)

In [ ]:
def test(f):
    return f.rand_property + 1

In [ ]:
foo = Foo(1000)
t = dask.delayed(test)(foo)

In [ ]:
t.compute()

In [ ]:
@dask.delayed
def compute_mean(array):
    return array.mean()
@dask.delayed
def mean_of_means(array_means):
    return np.array(array_means).mean()

In [ ]:
array_means = []
for _ in range(5):
    f = Foo(np.random.randint(10,10000))
    array_means.append(compute_mean(dask.delayed(f.rand_property)))
mean = mean_of_means(array_means)

In [ ]:
mean.compute()

In [ ]:
foo = Foo(1000)

In [ ]:
def get_dict_attr(obj, attr):
    for obj in [obj] + obj.__class__.mro():
        if attr in obj.__dict__:
            return obj.__dict__[attr]
    raise AttributeError

In [ ]:
def make_property_dask_lazy(instance,attr):
    for obj in [instance] + instance.__class__.mro():
        if attr in obj.__dict__:
            prop = obj.__dict__[attr]
            return dask.delayed(prop.fget)(instance)
    raise AttributeError

In [ ]:
test = get_dict_attr(foo,'rand_property')

In [ ]:
dtest = dask.delayed(test.fget)(foo)

In [ ]:
dtest.compute()

In [ ]:
test_loop = field.loops[0]

In [ ]:
electron_temperature = make_property_dask_lazy(field.loops[0],'electron_temperature')

In [ ]:
electron_temperature.compute()

In [ ]:
test_loop.electron_temperature.__